In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

W0716 15:23:36.042653 12700 deprecation.py:323] From <ipython-input-1-df6c3c653ca2>:3: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0716 15:23:36.044626 12700 deprecation.py:323] From c:\python36\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0716 15:23:36.045590 12700 deprecation.py:323] From c:\python36\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./mnist/data/train-images-idx3-ubyte.gz


W0716 15:23:39.615062 12700 deprecation.py:323] From c:\python36\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0716 15:23:39.621097 12700 deprecation.py:323] From c:\python36\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz


W0716 15:23:40.498244 12700 deprecation.py:323] From c:\python36\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
# 2차원 구성으로 직관적, X의 첫차원은 입력 데이터 개수, 마지막 1은 특징 개수, 데이터가 회색조라 색깔 하나뿐이라 1
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
# 출력값 10개 분류
Y = tf.placeholder(tf.float32, [None, 10])
# dropout위한 플레이스 홀더
keep_prob = tf.placeholder(tf.float32)

In [3]:
# 3x3 커널 가진 컨볼루션 계층 제작, 오른쪽과 아래쪽으로 한칸씩 움직이는 32개 커널 가진 컨볼루션 계층
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
# padding은 커널 슬라이딩 시 이미지의 가장 외곽에서 한칸 밖으로 움직이는 옵션
L1 = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding='SAME')
# 활성함수, 컴볼루션 완성
L1 = tf.nn.relu(L1)

In [4]:
# 풀링계층 만들기, 앞에 만큼 컨볼루션 계층을 입력으로 사용, 커널 크기 2x2인 풀링 계층
# strides는 승라이딩 시 두 칸씩 움직이겠다는 옵션
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [5]:
# 3x3커널 64개로 구성한 컨볼루션 계층과 2x2크기의 풀링 계층으로 구성
# 32는 앞서 구성한 첫번째 컨볼루션 계층의 커널 개수
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides = [1,2,2,1], padding='SAME')

In [6]:
# 추출한 특징으로 10개의 분류를 만들어내는 계층 구성
# 10개 분류는 1차원, 차원 줄이기부터! 직전 풀링이 7*7*64이므로 그크기의 1차원 계층 제작
# 배열 전체를 최종 출력값의 중간 단계인 256개 뉴럼으로 연결하는 신경망 제작
W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
# 과적합 막기 dropout 사용
L3 = tf.nn.dropout(L3, keep_prob)

W0716 16:42:22.085217 12700 deprecation.py:506] From <ipython-input-6-eea2aaed2f8c>:9: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
# 직전 은닉층 출력값 256개 받아 0~9출력 값인 10개 출력값 제작
W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

In [8]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
# 추후 optimizer = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)를 써서 비교해보자
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [12]:
# 학습과 결과 코드 작성, 앞과 큰 차이 없고 모델에 입력값을 전달하기 위해 28x28로 재구성 하는 부분만 좀 달라
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _,cost_val = sess.run([optimizer, cost],
                             feed_dict = {X:batch_xs,
                                         Y:batch_ys,
                                         keep_prob:0.7})
        total_cost += cost_val
    print('Epoch:', '%04d' % (epoch + 1),
         'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))
    
print('최적화 완료!')

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:', sess.run(accuracy,
                      feed_dict={X:mnist.test.images.reshape(-1, 28, 28, 1),
                                Y:mnist.test.labels,keep_prob: 1}))

Epoch: 0001 Avg. cost = 0.347
Epoch: 0002 Avg. cost = 0.110
Epoch: 0003 Avg. cost = 0.077
Epoch: 0004 Avg. cost = 0.059
Epoch: 0005 Avg. cost = 0.048
Epoch: 0006 Avg. cost = 0.041
Epoch: 0007 Avg. cost = 0.036
Epoch: 0008 Avg. cost = 0.030
Epoch: 0009 Avg. cost = 0.027
Epoch: 0010 Avg. cost = 0.025
Epoch: 0011 Avg. cost = 0.021
Epoch: 0012 Avg. cost = 0.021
Epoch: 0013 Avg. cost = 0.020
Epoch: 0014 Avg. cost = 0.015
Epoch: 0015 Avg. cost = 0.015
최적화 완료!
정확도: 0.9906


In [13]:
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides = [1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')
# L1 = tf.layers.conv2d(X, 32, [3, 3], activation = nn.tf.relu,padding='SAME')
# L1 = tf.layers.max_pooling2d(L1, [2, 2], [2, 2], padding='SAME')
# 이 두줄로 줄일 수 있다.

In [ ]:
L2 = tf.layer